### Import Dependencies

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sklearn as sk

### Load Dataset

In [2]:
trans = pd.read_excel('2022_Service.xlsx')
trans.head()

,State/Parent NTD ID,NTD ID,Agency Name,Reporter Type,Subrecipient Type,Reporting Module,Mode,TOS,Time Period,Time Service Begins,...,ADA UPT,Sponsored Service UPT,Passenger Miles,Days of Service Operated,Days Not Operated Due to Strikes,Strike Comment,Days Not Operated Due to Emergencies,Emergency Comment,Non-Statutory Mixed Traffic,DRM Mixed Traffic ROW
0,NaN,1,King County Department of Metro Transit,Full Reporter,NaN,Urban,DR,PT,Average Weekday - AM Peak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,King County Department of Metro Transit,Full Reporter,NaN,Urban,DR,PT,Average Weekday - Midday,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,King County Department of Metro Transit,Full Reporter,NaN,Urban,DR,PT,Average Weekday - PM Peak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1,King County Department of Metro Transit,Full Reporter,NaN,Urban,DR,PT,Average Weekday - Other,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1,King County Department of Metro Transit,Full Reporter,NaN,Urban,DR,PT,Average Typical Weekday,00:00:00,...,NaN,NaN,18431.0,255.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
row, col = trans.shape
print(f'Our dataset has {row} rows and {col} columns')

Our dataset has 13005 rows and 41 columns


### Visual Inspection
Here, we aim to discover key components of the dataset. 
* How many missing values do we have? (It seems like a lot!)
* What are the features of the dataset?
* And many more questions...       

From this initial inspection, we can better understand how to proceed.

#### Shorten column names

In [4]:
# define function for reusability
def get_cols(df):

    """
    @param df: dataframe you want the column of
    @type df: pandas DataFrame

    @return: tuple containing a list and string format of columns
    @return type: (list, string)
    """

    ft = df.columns.tolist()

    ft_s = '\n'
    # print features in easily readible format
    for i in range(len(ft)):
        if (i + 1) % 3 == 0:
            ft_s += ft[i] + ',\n'
        else:
            ft_s += ft[i] + ', '
    # strip last comma
    ft_s = ft_s.rstrip(', ')

    return ft, ft_s

In [5]:
old_features_lst, old_features_str = get_cols(trans)
print(f'Our dataframe has the following features: {old_features_str}')

Our dataframe has the following features: 
State/Parent NTD ID, NTD ID, Agency Name,
Reporter Type, Subrecipient Type, Reporting Module,
Mode, TOS, Time Period,
Time Service Begins, Time Service Ends, Vehicles/Passenger Cars Operated in Maximum Service,
Vehicles/Passenger Cars Available for Maximum Service, Trains in Operation, Vehicles/Passenger Cars in Operation,
Actual Vehicles/Passenger Car Miles, Actual Vehicles/Passenger Car Revenue Miles, Actual Vehicle/Passenger Deadhead Miles,
Scheduled Actual Vehicle/Passenger Car Revenue Miles, Actual Vehicle/Passenger Car Hours, Actual Vehicle/Passenger Car Revenue Hours,
Actual Vehicle/Passenger Car Deadhead Hours, Charter Service Hours, School Bus Hours,
Train Miles, Train Revenue Miles, Train Deadhead Miles,
Train Hours, Train Revenue Hours, Train Deadhead Hours,
Unlinked Passenger Trips (UPT), ADA UPT, Sponsored Service UPT,
Passenger Miles, Days of Service Operated, Days Not Operated Due to Strikes,
Strike Comment, Days Not Operated Du

In [6]:
# rename unnecessarily long column names, but ensure to retain meaning
trans = trans.rename(columns={'Vehicles/Passenger Cars Operated in Maximum Service' : 'Vehicles Operated in Max Service', 
                      'Vehicles/Passenger Cars Available for Maximum Service' : 'Vehicles Available for Max Service',
                      'Vehicles/Passenger Cars in Operation' : 'Vehicles in Operation',
                      'Actual Vehicles/Passenger Car Miles' : 'Actual Miles',
                      'Actual Vehicles/Passenger Car Revenue Miles' : 'Actual Revenue Miles',
                      'Actual Vehicle/Passenger Deadhead Miles' : 'Actual Deadhead Miles',
                      'Scheduled Actual Vehicle/Passenger Car Revenue Miles' : 'Scheduled Actual Revenue Miles',
                      'Actual Vehicle/Passenger Car Hours' : 'Actual Hours',
                      'Actual Vehicle/Passenger Car Revenue Hours' : 'Actual Revenue Hours',
                      'Actual Vehicle/Passenger Car Deadhead Hours' : 'Actual Deadhead Hours',
                      'Days of Service Operated' : 'Days Operated',
                      'Days Not Operated Due to Strikes' : 'Days Not Operated (Strikes)',
                      'Days Not Operated Due to Emergencies' : 'Days Not Operated (Emergencies)'})

In [7]:
# lets see our changes in action
features_lst, features_str = get_cols(trans)

print(f'Our dataframe manipulation resulted in {len(old_features_str) - len(features_str)} less characters in our columns!')

Our dataframe manipulation resulted in 226 less characters in our columns!


#### Deal with NaN values

In [14]:
nan_counts = trans.groupby('Agency Name').apply(lambda x: x.isna().sum(), include_groups=False)
nan_counts = nan_counts.sum()
nan_counts


State/Parent NTD ID                   11375
NTD ID                                    0
Reporter Type                             0
Subrecipient Type                     11375
Reporting Module                          0
Mode                                    100
TOS                                     100
Time Period                               0
Time Service Begins                    8103
Time Service Ends                      8107
Vehicles Operated in Max Service       9319
Vehicles Available for Max Service    11688
Trains in Operation                   12265
Vehicles in Operation                  9424
Actual Miles                           8361
Actual Revenue Miles                   5830
Actual Deadhead Miles                  8361
Scheduled Actual Revenue Miles        10358
Actual Hours                           8364
Actual Revenue Hours                   5935
Actual Deadhead Hours                  8364
Charter Service Hours                 12276
School Bus Hours                

In [18]:
percent_missing = trans.isnull().sum() * 100 / len(trans)
percent_missing

State/Parent NTD ID                    87.466359
NTD ID                                  0.000000
Agency Name                             0.000000
Reporter Type                           0.000000
Subrecipient Type                      87.466359
Reporting Module                        0.000000
Mode                                    0.768935
TOS                                     0.768935
Time Period                             0.000000
Time Service Begins                    62.306805
Time Service Ends                      62.337562
Vehicles Operated in Max Service       71.657055
Vehicles Available for Max Service     89.873126
Trains in Operation                    94.309881
Vehicles in Operation                  72.464437
Actual Miles                           64.290657
Actual Revenue Miles                   44.828912
Actual Deadhead Miles                  64.290657
Scheduled Actual Revenue Miles         79.646290
Actual Hours                           64.313725
Actual Revenue Hours